In [1]:
# ## Function to compare the locations and see if they are the same using the geolocation library in python
def location_check(slat, slon, dlat, dlon):
    import pygeohash as gh
    source = gh.encode(slat,slon,precision=5)
    destination = gh.encode(dlat,dlon,precision=5)
    if source == destination:
        return True
    else:
        return False

In [2]:
# Function to find the average in python
def find_average(var1, var2):
    return ((var1 + var2)/2)

In [3]:
def join_streams(stream1, stream2):
    if (stream1['latitude'] == stream2['latitude']) and (stream1['longitude'] == stream2['longitude']):
        stream = {}
        stream['latitude'] = stream1['latitude']
        stream['longitude'] = stream1['longitude']
        stream['air_temperature_celcius'] = stream1['air_temperature_celcius']
        stream['relative_humidity'] = stream1['relative_humidity']
        stream['windspeed_knots'] = stream1['windspeed_knots']
        stream['max_wind_speed'] = stream1['max_wind_speed']
        stream['precipitation'] = stream1['precipitation']
        stream['confidence'] = stream2['confidence']
        stream['surface_temperature_celcius'] =  stream2['surface_temperature_celcius']
        return stream

In [4]:
def avg_streams(stream1, stream2):
    stream = {}
    if(location_check(stream1['latitude'],stream2['longitude'],stream2['latitude'],stream2['longitude'])):
        stream['latitude'] = stream1['latitude']
        stream['longitude'] = stream['longitude']
        stream['confidence'] = find_average(stream1['confidence'], stream2['confidence'])
        stream['surface_temperature_celcius'] = find_average(stream1['surface_temperature_celcius'], stream2['surface_temperature_celcius'])
        return stream
    else:
        return stream

In [5]:
import os
import random
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    client = MongoClient()
    db = client.fit5148_assignment_db
    climate_data = db.climate_data
    hotspot_data = db.hotspot_data
    
#     key_list []
#     for record in iter:
#         key = record[0]
#         key_list.append(key)
        
#     if key_list[0] == key_list[1]:
#         # coming frm same tabke
        
#     else:
#         #coming from different table
        
    tableData = {} 
    for i, record in enumerate(iter):
        key = record[0]
        data = json.loads(record[1])
        jsonData = {}
        for k, v in data.items():
            jsonData[k] = v
        key = key + str(i)
        tableData[key] = jsonData
        
    # if the table is same then if it is climate then push to climate collection, else push it to hotspot collection
    
    # if location not same 
    # if one loc = aqua and other loc =terra > then 
            # check location 
            # if loc is same > find avg of confidence and surface temp and make it one data and push it to hostpot
            # if loc not same > discard
    # if loc = climate and other loc = either aqua/terra then 
    #the data of climate will go to climate collection and other data will go to hotspot collection.
    

    
    

    try:
        if(len(tableData) != 0):
            if len(tableData) == 1:
                if list(tableData.keys())[0][0:-1] == 'climate_data':
                    climate_data.insert_one(list(tableData.values())[0])
                else:
                    hotspot_data.insert_one(list(tableData.values())[0])
        
        else:
            if list(tableData.keys())[0][0:-1] == list(tableData.keys())[1][0:-1]:
                if list(tableData.keys())[0][0:-1] == "climate_data":
                    climate_data.insert_one(list(tableData.values())[0])
                    climate_data.insert_one(list(tableData.values())[1])
                else:
                    hotspot_data.insert_one(list(tableData.values())[0])
                    hotspot_data.insert_one(list(tableData.values())[1])
            else:
                if (list(tableData.keys())[0][0:-1] == "aqua_data" or list(tableData.keys())[0][0:-1] == "terra_data") & (list(tableData.keys())[1][0:-1] == "aqua_data" or list(tableData.keys())[1][0:-1] == "terra_data"):
                    lat1 = list(tableData.values())[0]['latitude']
                    lon1 = list(tableData.values())[0]['longitude']
                    lat2 = list(tableData.values())[1]['latitude']
                    lon2 = list(tableData.values())[1]['longitude']
                    if location_check(lat1,lon1,lat2,lon2):
                        jsonData = {}
                        jsonData['confidence'] = find_average(list(tableData.values())[0]['confidence'], list(tableData.values())[1]['confidence'])
                        jsonData['surface_temperature_celcius'] = find_average(list(tableData.values())[0]['surface_temperature_celcius'], list(tableData.values())[1]['surface_temperature_celcius'])
                        jsonData['latitude'] = lat1
                        jsonData['longitude'] = lon1
                        hotspot_data.insert_one(jsonData)
                else:
                    if list(tableData.keys())[0][0:-1] == 'climate_data':
                        climate_data.insert_one(list(tableData.values())[0])
                        hotspot_data.insert_one(list(tableData.values())[1])
                    else:
                        climate_data.insert_one(list(tableData.values())[1])
                        hotspot_data.insert_one(list(tableData.values())[0])
                        
            
        
        
        
        
        
#         if(len(tableData) != 0):
#             if len(tableData) == 1:
#                 if list(dic.keys())[0][0:-1] == 'climate_data':
#                     climate_data.insert_one(tableData)
#                 #else: do later
#             else:
#                 lat1 = list(tableData.values())[0]['latitude']
#                 lon1 = list(tableData.values())[0]['longitude']
#                 lat2 = list(tableData.values())[1]['latitude']
#                 lon2 = list(tableData.values())[1]['longitude']
#                 if location_check(lat1,lon1,lat2,lon2):
#                     #same location
#                     a = list(tableData.values())[0]
#                     b = list(tableData.values())[1]
#                     a.update(b)
#                     climate_data.insert_one(a)
#                 #else: 
#                     #diff location

    #climate_data.replace_one({"_id":data.get("ID")}, jsonData, True)
    except Exception as ex:
        print("Exception Occured. Message: {0}".format(str(ex)))
    client.close()

n_secs = 10
topic1 = "climate_streaming"
topic2 = "hotspot_streaming"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic1,topic2], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'climate-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)